In [3]:
import os
from torch.utils.data import Dataset
from torchvision.io import read_image
import vcd.core as core
import pandas as pd

In [12]:
class LidarDataset(Dataset):
    def __init__(self, dir = "lidar", path = "/vol/coro/automatic_train_operation/OSDaR23/", transform=None, target_transform=None, train: bool = True):
        self.train = train
        self.path = path
        self.dir = dir
        self.transform = transform
        self.target_transform = target_transform
        self.stations = self.data_search()
        self.df = []
        self.images = []
        self.targets = []
        self.classes = []
        self.cuboids = []
        # lidar_files = sorted(os.listdir(lidar_path))
        # irgendwo noch einen train test split einfügen.

        for datapath, labels_json in self.stations:
            self.data = self.data_list(datapath, labels_json)
            self.df.append(self.data)

        for station in self.df:
            for file in station[0]:
                self.images.append(file)
            for classes in station[1]:
                self.classes.append(classes)
            for cuboids in station[2]:
                self.cuboids.append(cuboids)
        self.targets.append(self.classes)
        self.targets.append(self.cuboids) 



    def data_search(self):
        path_list = []
        for station in os.listdir(self.path):
            path_final = os.path.join(self.path, station, self.dir)
            if station[0] == "0":
                station_w_ending = station[1:] + "_labels.json"
            else:
                station_w_ending = station + "_labels.json"
            label_frames = os.path.join(self.path, station, station_w_ending)
            path_list.append([path_final, label_frames])
        return path_list
    
    def data_list(self, data_path, labels_json):
        """
        Get list of filenames and coordinations from the cubiod.
        """
        # Create empty vcd object
        myVCD = core.VCD()
        # Load a VCD file
        myVCD.load_from_file(labels_json)
        frames_list_json = myVCD.data['openlabel']['frames']
        classes_list = []
        cuboid_list = []
        filenames =[]
        lidar_files = sorted(os.listdir(data_path))
        for lidar_file, frame in zip(lidar_files, frames_list_json):
            cuboid = []
            classes = []
            for objects in frames_list_json[frame]['objects']:
                object_data = frames_list_json[frame]['objects'][objects]['object_data']
                if 'cuboid' in object_data:
                    cuboid.append(frames_list_json[frame]['objects'][objects]['object_data']['cuboid'][0]['val'])
                    classes.append(frames_list_json[frame]['objects'][objects]['object_data']['cuboid'][0]['name'])
            cuboid_list.append(cuboid)
            classes_list.append(classes)
            if lidar_file.startswith(str(frame)) or lidar_file.startswith("0" + str(frame)) or lidar_file.startswith("00" + str(frame)):
                filenames.append(self.path + self.dir + "/" + str(lidar_file))
            else:
                print(lidar_file, frame)
        return [filenames, classes_list, cuboid_list]


    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index: int):
        """Get item at a given index.

        Args:
            index (int): Index
        Returns:
            tuple: (image, target), where
            target is a list of dictionaries with the following keys:

            'class': str,
            'cen_bb_x': float
            'cen_bb_y': float
            'cen_bb_z': float
            'vec_x': float
            'vec_y': float
            'vec_z': float
            'rota': float
            'bb_x': float
            'bb_y': float
            'bb_z': float

        """
        image = Image.open(self.images[index])
        target = self._parse_target(index) if self.train else None
        # if self.transforms:
        #     image, target = self.transforms(image, target)
        return image, target


    def _parse_target(self, index: int):
        target = []
        for cla, cuboid in zip(self.targets[0][index], self.targets[1][index]):
            target.append(
                {
                    'class': cla,
                    'cen_bb_x': cuboid[0],
                    'cen_bb_y': cuboid[1],
                    'cen_bb_z': cuboid[2],
                    'vec_x': cuboid[3],
                    'vec_y': cuboid[4],
                    'vec_z': cuboid[5],
                    'rota': cuboid[6],
                    'bb_x': cuboid[7],
                    'bb_y': cuboid[8],
                    'bb_z': cuboid[9]
                }
            )
        return target

In [13]:
a = LidarDataset()

In [14]:
a[4]

NameError: name 'Image' is not defined

In [ ]:
class LidarDataset(Dataset):
    def __init__(self, dir = "lidar", path = "/vol/coro/automatic_train_operation/OSDaR23/", transform=None, target_transform=None):
        # Labels
        # self.img_labels = pd.read_csv(annotations_file)

        self.path = path
        self.dir = dir
        self.transform = transform
        self.target_transform = target_transform
        self.labels = self.data_search()
        self.df = []
        for i in self.labels:
            self.train_df = self.generate_train_df(i[0], i[1])
            self.df.append(self.train_df)
        self.annotations = [item for subl in self.df for item in subl]
        # self.annotations = pd.DataFrame(self.annotations)
        # print(len(self.annotations))


    def data_search(self):
        path_list = []
        for station in os.listdir(self.path):
            path_final = os.path.join(self.path, station, self.dir)
            if station[0] == "0":
                station_w_ending = station[1:] + "_labels.json"
            else:
                station_w_ending = station + "_labels.json"
            label_frames = os.path.join(self.path, station, station_w_ending)
            path_list.append([path_final, label_frames])
        return path_list
    
    def generate_train_df (self, lidar_path, frames_path):
        # Create empty vcd object
        myVCD = core.VCD()
        # Load a VCD file
        myVCD.load_from_file(frames_path)
        frames_list = myVCD.data['openlabel']['frames']

        anno_list = []    
        lidar_files = sorted(os.listdir(lidar_path))
        for lidar_file, frame in zip(lidar_files, frames_list):
            for objects in frames_list[frame]['objects']:
                object_data = frames_list[frame]['objects'][objects]['object_data']
                if 'cuboid' in object_data:
                    cuboid = frames_list[frame]['objects'][objects]['object_data']['cuboid'][0]['val']
                    classes = frames_list[frame]['objects'][objects]['object_data']['cuboid'][0]['name']
                    anno = {}
                    if lidar_file.startswith(str(frame)) or lidar_file.startswith("0" + str(frame)) or lidar_file.startswith("00" + str(frame)):
                        anno['filename'] = self.path + self.dir + "/" + str(lidar_file)
                    else:
                        print(lidar_file, frame)
                    anno['class'] = classes
                    anno['cen_bb_x'] = cuboid[0]
                    anno['cen_bb_y'] = cuboid[1]
                    anno['cen_bb_z'] = cuboid[2]
                    anno['vec_x'] = cuboid[3]
                    anno['vec_y'] = cuboid[4]
                    anno['vec_z'] = cuboid[5]
                    anno['rota'] = cuboid[6]
                    anno['bb_x'] = cuboid[7]
                    anno['bb_y'] = cuboid[8]
                    anno['bb_z'] = cuboid[9]
                    anno_list.append(anno)
        return anno_list

        # # Image
        # path_list = []
        # for station in os.listdir(self.path):
        #     path_final = os.path.join(self.path, station, self.img_dir)

        #     for filename in os.listdir(path_final):
        #         path_filename = os.path.join(path_final, filename)

        #         # # checking if it is a file
        #         # if os.path.isfile(path_filename):
        #         #     print(path_filename)
        #         path_list.append(path_filename)


        # return path_list

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.path, )
        # img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [6]:
LidarDataset()

26744
